# Algoritma 17.4 (Bound-Constrained Lagrangian Method)

In [2]:
import numpy as np

def projection(x, l, u):
    """Projects x onto the box constraints l <= x <= u."""
    return np.maximum(l, np.minimum(x, u))

def bound_constrained_lagrangian(x0, lambd0, c_func, grad_L_func, l, u, 
                                  eta_tol=1e-6, omega_tol=1e-6, mu0=10):
    """Implements Algorithm 17.4: Bound-Constrained Lagrangian Method."""
    
    mu = mu0
    omega = 1 / mu0
    eta = 1 / mu0**0.1
    x = x0.copy()
    lambd = lambd0.copy()
    
    for k in range(1000):  # Set a reasonable iteration limit
        # Solve subproblem
        grad_L = grad_L_func(x, lambd, mu)
        x_new = projection(x - grad_L, l, u)
        
        # Check stationarity condition
        if np.linalg.norm(x_new - projection(x - grad_L, l, u)) <= omega:
            if np.linalg.norm(c_func(x_new)) <= eta:
                # Convergence test
                if np.linalg.norm(c_func(x_new)) <= eta_tol and np.linalg.norm(x_new - projection(x - grad_L, l, u)) <= omega_tol:
                    return x_new  # Stop with approximate solution
                
                # Update multipliers and tighten tolerances
                lambd -= mu * c_func(x_new)
                mu = mu
                eta /= mu**0.9
                omega /= mu
            else:
                # Increase penalty parameter and tighten tolerances
                mu *= 100
                eta = 1 / mu**0.1
                omega = 1 / mu
        
        x = x_new  # Update x
    
    return x  # Return last iterate if max iterations reached
